# Generic NN trainer

In [2]:
import mlflow
import mlflow.keras
from numpy import ndarray
import pandas as pd
import sys

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow as tf

%load_ext lab_black

sys.path.append("/Users/renero/Documents/SideProjects/trader/src")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/predictor")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/utils")

from cs_dictionary import CSDictionary
from lstm import lstm_1layer, lstm_2layer
from sequences import sequences
from ticks import Ticks
from metrics import metrics

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [ ]:
argv = [
    "",
    "-c",
    "../src/predictor/params.dax.yaml",
    "-f",
    "../data/^GDAXI/dax_2018_2019.csv",
    "--window",
    "8",
    "--epochs",
    "400",
    "train",
]
params = CSDictionary(args=argv)

ticks = Ticks(params, params.input_file, scale=True)

X_train, y_train, X_test, y_test = sequences.prepare(
    ticks.data, timesteps=params.window_size, test_size=params.test_size
)

params.num_features = sequences.get_num_features(X_train)
params.num_target_labels = sequences.get_num_target_labels(y_train)

nn = lstm_2layer(params)

nn.start_experiment(X_train, y_train, "2layers")
result = nn.evaluate_experiment(X_test, y_test)
nn.end_experiment()

In [ ]:
metrics.plot_and_compare(result)